In [1]:
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train_text = train['comment_text']
test_text = test['comment_text']
vec = TfidfVectorizer(max_features=50000).fit(train_text)
X_train = vec.transform(train_text)
X_test = vec.transform(test_text)

In [3]:
cross_val_scores = []
predictions = {'id': test['id']}
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for class_name in classes:
    y_train = train[class_name]
    clf = LogisticRegression(random_state=0)
    cross_val_scores.append(np.mean(cross_val_score(clf, X_train, y_train, scoring='roc_auc')))
    clf.fit(X_train, y_train)
    predictions[class_name] = clf.predict_proba(X_test)[:, 1]
print("cross validation score :", np.mean(cross_val_scores))
submission = pd.DataFrame(predictions, columns=["id"] + classes)
submission.to_csv('submission/v1_TFIDF_LR_submission.csv.gz', compression="gzip", index=False)

cross validation score : 0.9781988175354064
